# 1、项目背景

使用SVM和ARIMA进行股价预测
使用Tushare库，获取平安银行（000001）2019年5月12至2019年12月19的股票数据

# 2、数据导入

In [1]:
import tushare as ts
import pandas as pd  
import numpy as np
from pyecharts import Kline,Line, Bar,Overlap

2022-06-09 20:13:58,519 - lml.utils - ERROR - failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "E:\Anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
def date_setting(stock_code, start_date, end_date):
    global tsData, stockCode

    stockCode = stock_code
    tsData = ts.get_hist_data(code=stock_code, start=start_date, end=end_date)
    tsData = tsData.sort_index(ascending=True).reset_index()

    if len(tsData) != 0:
        print('股票数据已成功获取')
        tsData
    else:
        print('股票数据获取失败！')

In [3]:
date_setting(stock_code='000001',start_date='2019-05-12', end_date='2019-12-19')

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
股票数据已成功获取


In [4]:
tsData

,date,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
0,2019-12-10,15.36,15.38,15.33,15.25,556383.88,-0.08,-0.52,15.330,15.330,15.330,556383.88,556383.88,556383.88,0.29
1,2019-12-11,15.38,15.68,15.66,15.30,871505.19,0.33,2.15,15.495,15.495,15.495,713944.54,713944.54,713944.54,0.45
2,2019-12-12,15.66,15.67,15.60,15.53,478998.81,-0.06,-0.38,15.530,15.530,15.530,635629.29,635629.29,635629.29,0.25
3,2019-12-13,15.85,16.12,16.12,15.77,1053103.88,0.52,3.33,15.678,15.678,15.678,739997.94,739997.94,739997.94,0.54
4,2019-12-16,16.12,16.14,16.13,15.87,787952.31,0.01,0.06,15.768,15.768,15.768,749588.81,749588.81,749588.81,0.41
5,2019-12-17,16.00,16.63,16.50,15.98,1203104.88,0.37,2.29,16.002,15.890,15.890,878933.01,825174.83,825174.83,0.62
6,2019-12-18,16.43,16.66,16.46,16.41,797091.38,-0.04,-0.24,16.162,15.971,15.971,864050.25,821162.90,821162.90,0.41
7,2019-12-19,16.55,16.74,16.55,16.44,675536.81,0.09,0.55,16.352,16.044,16.044,903357.85,802959.64,802959.64,0.35


In [5]:
#绘制K线图
def kline_plot(df):
        #画K线图数据
        df.index=pd.to_datetime(df.date)
        date = df.index.strftime('%Y%m%d').tolist()
        k_value = df[['open','close', 'low','high']].values
        #引入pyecharts画图使用的是0.5.11版本，新版命令需要重写
        
        kline = Kline()
        kline.add('日K线图', date, k_value,
              is_datazoom_show=True,is_splitline_show=False)
        #加入5、20日均线
        df['ma20']=df.close.rolling(20).mean()
        df['ma5']=df.close.rolling(5).mean()
        line = Line()
        v0=df['ma5'].round(2).tolist()
        v=df['ma20'].round(2).tolist()
        line.add('5日均线', date,v0,
             is_symbol_show=False,line_width=2)
        line.add('20日均线', date,v, 
             is_symbol_show=False,line_width=2)
        #成交量
        bar = Bar()
        bar.add('成交量', date, df['volume'],tooltip_tragger='axis', 
                is_legend_show=False, is_yaxis_show=False, 
                yaxis_max=5*max(df['volume']))
        overlap = Overlap()
        overlap.add(kline)
        overlap.add(line,)
        overlap.add(bar,yaxis_index=1, is_add_yaxis=True)
        return overlap    

In [6]:
kline_plot(tsData)

# 3、SVM预测股价上涨

In [7]:
import pandas as pd
from sklearn import svm,preprocessing
import tushare as ts

In [8]:
class SVM_Predict:
    stock_code = ''
    tsData = pd.DataFrame()
    def __init__(self, stock_code):
        self.stock_code = stock_code
    def date_setting(self, start_date, end_date):
        self.tsData = ts.get_hist_data(code=self.stock_code, start=start_date, end=end_date)
        self.tsData = self.tsData.reset_index()
    def makeSVMPrediction(self, rate): # rate表示训练集和测试集的比例
        df_CB = self.tsData.sort_index(ascending=True, axis=0)
        df_CB = df_CB.set_index('date')
        df_CB = df_CB.sort_index()
        # value表示涨跌, =1为涨，=0为跌
        value = pd.Series(df_CB['close'] - df_CB['close'].shift(1), \
                          index=df_CB.index)#相比昨天涨跌幅度 ，shift(1)是把数据向下移动1位
        value = value.bfill()# 用于填充给定对象中的缺失值。缺失数据比较多的情况下，可以直接滤除，缺失数据比较少时，对数据进行填充就很有必要了。
        value[value >= 0] = 1
        value[value < 0] = 0
        df_CB['Value'] = value
        # 后向填充空缺值
        df_CB = df_CB.fillna(method='bfill')
        df_CB = df_CB.astype('float64')
        print(df_CB.head())

        L = len(df_CB)
        train = int(L * rate)
        total_predict_data = L - train

        # 对样本特征进行归一化处理
        df_CB_X = df_CB.drop(['Value'], axis=1)
        df_CB_X = preprocessing.scale(df_CB_X)

        # 开始循环预测，每次向前预测一个值
        correct = 0
        train_original = train
        while train < L:
            Data_train = df_CB_X[train - train_original:train]
            value_train = value[train - train_original:train]
            Data_predict = df_CB_X[train:train + 1]
            value_real = value[train:train + 1]

            # 核函数分别选取'ploy','linear','rbf'
            # classifier = svm.SVC(C=1.0, kernel='poly')
            # classifier = svm.SVC(kernel='linear')
            classifier = svm.SVC(C=1.0, kernel='rbf')
            classifier.fit(Data_train, value_train)
            value_predict = classifier.predict(Data_predict)
            print("value_real=%d value_predict=%d" % (value_real[0], value_predict))
            # 计算测试集中的正确率
            if (value_real[0] == int(value_predict)):
                correct = correct + 1
            train = train + 1
        print(correct)
        print(total_predict_data)
        correct = correct * 100 / total_predict_data
        print("Correct=%.2f%%" % correct)

a = SVM_Predict('000001')
a.date_setting(start_date='2019-05-12', end_date='2019-12-19')
a.makeSVMPrediction(0.8)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
             open   high  close    low      volume  price_change  p_change  \
date                                                                         
2019-12-10  15.36  15.38  15.33  15.25   556383.88         -0.08     -0.52   
2019-12-11  15.38  15.68  15.66  15.30   871505.19          0.33      2.15   
2019-12-12  15.66  15.67  15.60  15.53   478998.81         -0.06     -0.38   
2019-12-13  15.85  16.12  16.12  15.77  1053103.88          0.52      3.33   
2019-12-16  16.12  16.14  16.13  15.87   787952.31          0.01      0.06   

               ma5    ma10    ma20      v_ma5     v_ma10     v_ma20  turnover  \
date                                                                            
2019-12-10  15.330  15.330  15.330  556383.88  556383.88  556383.88      0.29   
2019-12-11  15.495  15.495  15.495  713944.54  713944.54  713944.54      0.45   
2019-12-12  15.530  15.530  15.530  635629.29  635629.29  635629.29      0.2

# 4、ARIMA预测股票价格

In [9]:
import pandas as pd
import tushare as ts
import matplotlib.pyplot as plt
from pmdarima import auto_arima

In [10]:
import matplotlib.pyplot as plt
class AutoARIMA_pridict:
    stock_code = ''
    tsData = pd.DataFrame()
    def __init__(self, stock_code):
        self.stock_code = stock_code
    def date_setting(self, start_date, end_date):
        self.tsData = ts.get_hist_data(code=self.stock_code, start=start_date, end=end_date)
        self.tsData = self.tsData.reset_index()
    def makePredictionByDay(self, node): # 按日回测
        new_data = pd.DataFrame(index=range(0, len(self.tsData)), columns=['Date', 'Close'])
        for i in range(0, len(self.tsData)):
            new_data['Date'][i] = self.tsData['date'][i]
            new_data['Close'][i] = self.tsData['close'][i]
        new_data['Date'] = pd.to_datetime(new_data.Date, format='%Y-%m-%d')
        new_data.index = new_data['Date']
        # 准备数据
        new_data = new_data.sort_index(ascending=True)
        forecast = []
        # 训练集和预测集
        prediction = new_data[node:]
        for i in range(0, len(new_data) - node):
            train = new_data[:node + i]
            valid = new_data[node + i:]
            # 对收盘价进行测试
            training = train_1['Close']
            validation = valid['Close']
            # 拟合模型
            model = auto_arima(training, start_p=1, start_q=1, max_p=2, max_q=2, m=12, start_P=0, seasonal=True, d=1,
                               D=1,
                               trace=True, error_action='ignore', suppress_warnings=True)  #
            model.fit(training)
            # 预测
            forecast.append(model.predict(n_periods=1)[0])
        prediction['Prediction'] = forecast
        plt.plot(train['Close'])
        plt.plot(prediction[['Close', 'Prediction']])
        plt.show()

In [11]:
a = AutoARIMA_pridict('000001')
a.date_setting(start_date='2019-05-12', end_date='2019-12-19')
a.makePredictionByDay(140)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


UnboundLocalError: local variable 'train_1' referenced before assignment